In [1]:
import pandas as pd
import numpy as np
import duckdb
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

import joblib

In [2]:
# Cargo los datos en un DataFrame
df = pd.read_csv('03_data_procesada/01_data_pro_2.csv')

Defino variables predictoras y target del conjunto de datos

In [3]:
numeric_features = ['enginesize', 'curbweight', 'horsepower', 'carwidth', 'citympg', 'carlength']
categorical_features = ['Brand', 'cylindernumber']

predictors = ['enginesize', 'curbweight', 'horsepower', 'carwidth', 'citympg', 'carlength', 'Brand', 'cylindernumber']
target = ['price']

Dada la Importancia que definimos tiene la Marca en la Predicción, me encargo de que la cantidad de Datos por Marca (`Brand`) se equivalente tanto para Train como para Test

In [4]:
X_test = pd.DataFrame(columns=predictors)
X_train = pd.DataFrame(columns=predictors)
y_test = pd.DataFrame(columns=target)
y_train = pd.DataFrame(columns=target)

brands = df['Brand'].unique()

for brand in brands:
    mask = df['Brand'] == brand
    X_brand = df.loc[mask][predictors]
    y_brand = df.loc[mask][target]
    if len(X_brand) > 1:
        X_brand_train, X_brand_test, y_brand_train, y_brand_test = train_test_split(X_brand, y_brand, test_size=0.5, random_state=42)
    else:
        X_brand_train = X_brand
        X_brand_test = X_brand
        y_brand_train = y_brand
        y_brand_test = y_brand    
    
    X_train = pd.concat([X_train, X_brand_train], axis=0)
    X_test = pd.concat([X_test, X_brand_test], axis=0)
    y_train = pd.concat([y_train, y_brand_train], axis=0)
    y_test = pd.concat([y_test, y_brand_test], axis=0)
    

 Crear el preprocesador para las variables categóricas

In [5]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

Crear el pipeline del modelo

In [6]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

Configurar la Grilla
- `n_estimators`: Número de árboles en el modelo.
- `learning_rate`: Tasa de aprendizaje.
- `max_depth`: Profundidad máxima de cada árbol.
- `subsample`: Proporción de muestras utilizadas para entrenar cada árbol.
- `colsample_bytree`: Proporción de características utilizadas para entrenar cada árbol.
- `gamma`: Reducción mínima de pérdida requerida para hacer una partición adicional en un nodo de árbol

In [7]:
# Aca se jugo con muchas combinaciones pero deje la combinación que mejor resultado arrojó

param_grid = {
    "regressor__n_estimators": [170],
    "regressor__learning_rate": [0.11],
    "regressor__max_depth": [5],
    "regressor__subsample": [0.75],
    "regressor__colsample_bytree": [0.65],
    "regressor__gamma": [1]
}

grid_search = GridSearchCV(
    model, param_grid, cv=5, scoring="neg_mean_squared_error", n_jobs=-1
)

 Entrenar y evaluar el modelo

In [8]:

# Aca se entrena el Modelo con Todas las combinaciones que se establezcan en la Grilla
grid_search.fit(X_train, y_train)

# Mejor combinación de hiperparámetros
best_params = grid_search.best_params_
print(f"Mejor combinación de hiperparámetros: {best_params}")

# Evaluar el modelo con los mejores hiperparámetros --> best_model es la Variable que contendrá el Modelo Entrenado
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
best_mse = mean_squared_error(y_test, y_pred_best)
best_r2 = r2_score(y_test, y_pred_best)

print(f"RMSE con mejores hiperparámetros: {np.sqrt(best_mse)}")
print(f"MSE con mejores hiperparámetros: {best_mse}")
print(f"R² con mejores hiperparámetros: {best_r2}")

# ******************************
print('*'*50)
y_pred_train = best_model.predict(X_train)
print(f"R² del Train: {r2_score(y_train, y_pred_train)}")
print(f"RMSE del Train: {np.sqrt(mean_squared_error(y_train, y_pred_train))}")

Mejor combinación de hiperparámetros: {'regressor__colsample_bytree': 0.65, 'regressor__gamma': 1, 'regressor__learning_rate': 0.11, 'regressor__max_depth': 5, 'regressor__n_estimators': 170, 'regressor__subsample': 0.75}
RMSE con mejores hiperparámetros: 2319.858632318536
MSE con mejores hiperparámetros: 5381744.073942827
R² con mejores hiperparámetros: 0.9188436269760132
**************************************************
R² del Train: 0.9995906949043274
RMSE del Train: 156.43241857824626



Toca Ahora crear una Medicion Propia mas entendible a lo que estamos estudiando

In [12]:
predicciones = best_model.predict(X_test)

resultados = pd.DataFrame()
resultados['y_test'] = y_test
resultados['y_pred'] = predicciones
resultados['Diferencia'] = resultados['y_test'] - resultados['y_pred']
resultados['Porcentaje'] = round((resultados['y_pred'] / resultados['y_test'] - 1),4) * 100

duckdb.sql("""SELECT * FROM resultados;""").show()

┌─────────┬───────────┬──────────────────┬─────────────────────┐
│ y_test  │  y_pred   │    Diferencia    │     Porcentaje      │
│ double  │   float   │      double      │       double        │
├─────────┼───────────┼──────────────────┼─────────────────────┤
│ 13495.0 │   13314.8 │   180.2001953125 │               -1.34 │
│ 16500.0 │   13314.8 │  3185.2001953125 │               -19.3 │
│ 13950.0 │   10354.9 │   3595.099609375 │              -25.77 │
│ 17450.0 │  16067.71 │  1382.2900390625 │  -7.920000000000001 │
│ 23875.0 │ 20309.777 │    3565.22265625 │              -14.93 │
│ 15250.0 │ 12192.531 │       3057.46875 │              -20.05 │
│ 16925.0 │ 10585.957 │    6339.04296875 │              -37.45 │
│ 30760.0 │  37469.81 │   -6709.80859375 │               21.81 │
│ 16430.0 │ 10585.957 │    5844.04296875 │              -35.57 │
│ 36880.0 │  41983.44 │   -5103.44140625 │               13.84 │
│     ·   │      ·    │          ·       │                  ·  │
│     ·   │      ·    │  

Calculamos la Sumatoria de los Valores Absolutos de los errores

In [13]:
duckdb.sql("""SELECT SUM(ABS(Porcentaje)) FROM resultados;""")

┌──────────────────────┐
│ sum(abs(Porcentaje)) │
│        double        │
├──────────────────────┤
│              1373.13 │
└──────────────────────┘

Calculamos la media del error 

In [14]:
promedio = round((1373.13/ resultados.shape[0]),2)
promedio

12.6

En Total hay una media de 12,6 % de error entre el Valor Predicho y la realidad de los datos. 

***
***
Guardando el Modelo

In [209]:
joblib.dump(best_model, 'models/modelo_proyecto_02.pkl')

['models/modelo_proyecto_02.pkl']

***
***
Preparandonos para trabajar en la App <br>
Valores que pueden adoptar las distintas Predictoras

In [ ]:
enginesizes = [61, 326]       #Valor Minimo y Valor Maximo
curbweights = [1488, 4066]    #Valor Minimo y Valor Maximo  
citympgs = [13, 49]           #Valor Minimo y Valor Maximo
carlenght = [141.1, 208.1]    #Valor Minimo y Valor Maximo
horspowers = [48, 288]        #Valor Minimo y Valor Maximo
carwidths = [60.3, 72.3]      #Valor Minimo y Valor Maximo      
brands = ['Bmw', 'Volvo', 'Plymouth', 'Mitsubishi', 'Buick', 'Subaru', 'Volkswagen', 'Toyota', 'Jaguar', 'Dodge', 'Mazda',
          'Porsche', 'Alfa-romeo', 'Audi', 'Nissan', 'Isuzu', 'Chevrolet', 'Mercury', 'Saab', 'Renault', 'Peugeot', 'Honda']
cylindernumbers = ['two',  'three', 'four', 'five', 'six', 'eight', 'twelve']

***
***
# Como se Probaría el Modelo en la App

In [1]:
import pandas as pd
import joblib

Cargamos los datos de Prueba en un diccionario y creamos el Dataframe de Prueba

In [5]:
# predictors = ['enginesize', 'curbweight', 'horsepower', 'carwidth', 'citympg', 'carlength', 'Brand', 'cylindernumber']
inputs = {
    'enginesize': 209, 
    'curbweight': 2548, 
    'horsepower': 182, 
    'carwidth': 66.9, 
    'citympg': 16, 
    'carlength': 189.0, 
    'Brand': 'Bmw', 
    'cylindernumber': 'six'
}

# inputs_list  = [120,3095,97,65.2,27,173.4, "Nissan", "four"]
valores_test = pd.DataFrame(inputs, index=[0])
valores_test

Cargando modelo

In [4]:
modelo_cargado = joblib.load('models/modelo_proyecto_02.pkl')

Realizamos la Predicción

In [8]:
prediccion = modelo_cargado.predict(valores_test)
prediccion[0]

32532.252

***
***
Preparando la data de la tabla de la app

In [10]:
# Cargaremos una Data de Ejemplo que tendra 2 registros por Marca para que sean Datos de Prueba de la App
tabla = pd.DataFrame(columns=df.columns)
brands = df['Brand'].unique()

for brand in brands:
    mask = df['Brand'] == brand
    if len(df.loc[mask]) > 1:
        temporal = df.loc[mask].sample(n=2)
    else: 
        temporal = df.loc[mask]  
    tabla = pd.concat([tabla, temporal], axis=0)

Salvamos la Data de Ejemplo que utilizaremos en la App

In [11]:
tabla = tabla[predictors + ['price']]

tabla.to_csv('04_data_ejemplo_tabla_app/tabla_proj_2.csv', index=False)